#### import some library

In [39]:
import easyocr
import keras
from keras import ops
from keras import layers
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

In [5]:
def extract_text(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    text = " "
    for (bbox, text_s, prob) in result:
        text += text_s
    return text

In [50]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, dtype='float32', **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [51]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, dtype='float32', **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


#### prepare dataset

In [43]:
# Load the dataset
dataset_path = "D:/Github/colab_ml_model_learning/dataset/tweet_emotions.csv"
dataset = pd.read_csv(dataset_path)

# Assuming 'content' column contains the text data and 'sentiment' column contains the labels
texts = dataset['content'].values
labels = dataset['sentiment'].values
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each tweet

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

In [44]:
print(set(y_train))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}


In [45]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
class_number = 12

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(class_number + 1, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [46]:
model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_5  │ (None, 200, 32)        │       646,400 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_5             │ (None, 200, 32)        │        10,656 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_5      │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 20)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 13)             │           273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 657,989 (2.51 MB)

 Trainable params: 657,989 (2.51 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)

Epoch 1/2
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 95s 87ms/step - accuracy: 0.1939 - loss: 2.2386 - val_accuracy: 0.2561 - val_loss: 2.1413
Epoch 2/2
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 154s 99ms/step - accuracy: 0.2378 - loss: 2.1380 - val_accuracy: 0.3194 - val_loss: 1.9708


#### Save sentiment_model

In [48]:
# Save the model
model.save("./model/sentiment_model.h5")

In [52]:
# Load the model
loaded_model = load_model("./model/sentiment_model.h5", custom_objects={"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock})

In [53]:
example_texts = [
    "I love this movie, it's amazing!",
    "This book is terrible, I regret buying it.",
    "The weather today is beautiful.",
    "I had a bad experience with their customer service.",
    "The food at that restaurant was delicious.",
    "I feel so happy after seeing my friends.",
    "The traffic is unbearable today.",
    "The new album by my favorite artist is disappointing.",
    "I'm excited about my upcoming vacation.",
    "I'm feeling stressed out because of work.",
]


In [54]:
preprocessed_examples = tokenizer.texts_to_sequences(example_texts)
padded_examples = pad_sequences(preprocessed_examples, maxlen=maxlen)

# Predict using the loaded model
predictions = loaded_model.predict(padded_examples)

# Convert predictions to labels
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))

print(predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 830ms/step
['love' 'worry' 'happiness' 'worry' 'neutral' 'happiness' 'worry'
 'neutral' 'love' 'worry']
